In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers

### Simulation (in Matrix-Representation)

Agents (NNs) are given bird persepective images of his square shaped world.
Their Action Space contains the Options staying, or moving (right, left, up, down).
Mit Hilfe von Linarer Algrebrah wird, dem output des Agents entsprechend, ein Foto ihrer Welt im nächsten Simulations-Schritt berechnet.
Nach S Simulations-Schritten wird die Position des Agents der Fitness Function übergeben.

The dimentions of the world:

In [93]:
L, W = 10, 10

Agent:

In [20]:
def get_agent_model():

    A = layers.Input(shape=(L,W,2))
    h = layers.Flatten()(A)
    h = layers.Dense(
        4, 
        activation='sigmoid',
        bias_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
    )(h)
    # h = tf.linalg.normalize(h)
    # h = layers.LayerNormalization()(h)  
    h = tf.math.divide(
        tf.math.subtract(
            h, 
            tf.reduce_min(h)
        ), 
        tf.math.subtract(
            tf.reduce_max(h), 
            tf.reduce_min(h)
        )
    )
    motion = tf.math.round(h)

    return tf.keras.Model(A, motion, name='agent_nn')

In [101]:
def get_enviroment_model(A, M, K):

    # New Position Vektor K acording to motion M

    M_1 = tf.math.scalar_mul(
            scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[1,0,0,0]], dtype=tf.float32))),
            x=tf.constant([[-1,0]], dtype=tf.float32),
            name='coord. step up'
        )

    M_2 = tf.math.scalar_mul(
            scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,1,0,0]], dtype=tf.float32))),
            x=tf.constant([[1,0]], dtype=tf.float32),
            name='coord. step down'
        )

    M_3 = tf.math.scalar_mul(
            scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,0,1,0]], dtype=tf.float32))),
            x=tf.constant([[0,-1]], dtype=tf.float32),
            name='coord. step left'
        )

    M_4 = tf.math.scalar_mul(
            scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,0,0,1]], dtype=tf.float32))),
            x=tf.constant([[0,1]], dtype=tf.float32),
            name='coord. step right'
        )

    M_12 = tf.math.add(M_1, M_2)
    M_34 = tf.math.add(M_3, M_4)
    M_all = tf.math.add(M_12, M_34)
    K_new = tf.math.add(K, M_all)



    # New image B according to motion M

    kernel = np.zeros(shape=(3,3,2,2))
    kernel[1,1,1,1] = 1                     # The convolution on the target dimention in the enviroment's image is the identity


    '''Up'''
    conv_layer_up = layers.Conv2D(filters=2, kernel_size=(3,3), padding='same', kernel_initializer=tf.constant_initializer(1.))
    conv_layer_up_output = conv_layer_up(A)
    
    kernel_up = kernel.copy()
    kernel_up[2,1,0,0] = 1 
    conv_layer_up.set_weights([tf.constant(kernel_up, dtype=tf.float32), tf.constant(0, shape=(2))])                                  

    A_1 = tf.math.scalar_mul(
        scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[1,0,0,0]], dtype=tf.float32))),
        x=conv_layer_up_output, 
        name='image_step_up_multi'
    )
    A_1 = tf.math.add(A_1, tf.math.scalar_mul(scalar=1 - tf.reduce_sum(tf.math.multiply(M, tf.constant([[1,0,0,0]], dtype=tf.float32))), x=A), name='image_step_up_add')

    '''Down'''
    conv_layer_down = layers.Conv2D(filters=2, kernel_size=(3,3), padding='same', kernel_initializer=tf.constant_initializer(1.))
    conv_layer_down_output = conv_layer_down(A_1)
    
    kernel_down = kernel.copy()
    kernel_down[0,1,0,0] = 1 
    conv_layer_down.set_weights([tf.constant(kernel_down, dtype=tf.float32), tf.constant(0, shape=(2))])

    A_2 = tf.math.scalar_mul(
        scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,1,0,0]], dtype=tf.float32))),
        x=conv_layer_down_output
    )
    A_2 = tf.math.add(A_2, tf.math.scalar_mul(scalar=1 - tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,1,0,0]], dtype=tf.float32))), x=A_1))

    '''Left'''
    conv_layer_left = layers.Conv2D(filters=2, kernel_size=(3,3), padding='same', kernel_initializer=tf.constant_initializer(1.))
    conv_layer_left_output = conv_layer_left(A_2)
    
    kernel_left = kernel.copy()
    kernel_left[1,2,0,0] = 1 
    conv_layer_left.set_weights([tf.constant(kernel_left, dtype=tf.float32), tf.constant(0, shape=(2))])

    A_3 = tf.math.scalar_mul(
        scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,0,1,0]], dtype=tf.float32))),
        x=conv_layer_left_output
    )
    A_3 = tf.math.add(A_3, tf.math.scalar_mul(scalar=1 - tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,0,1,0]], dtype=tf.float32))), x=A_2))

    '''Right'''
    conv_layer_right = layers.Conv2D(filters=2, kernel_size=(3,3), padding='same', kernel_initializer=tf.constant_initializer(1.))
    conv_layer_right_output = conv_layer_right(A_3)
    
    kernel_right = kernel.copy()
    kernel_right[1,0,0,0] = 1 
    conv_layer_right.set_weights([tf.constant(kernel_right, dtype=tf.float32), tf.constant(0, shape=(2))])

    A_4 = tf.math.scalar_mul(
        scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,0,0,1]], dtype=tf.float32))),
        x=conv_layer_right_output
    )
    A_new = tf.math.add(A_4, tf.math.scalar_mul(scalar=1 - tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,0,0,1]], dtype=tf.float32))), x=A_3))

    return tf.keras.Model((A, M, K), (A_new, K_new), name='Enviroment')

### Fitness Function (in Matrix-Representation)

Subtration der Target-Position von den x,y Koordianten

In [75]:
pop_num = 4
A = layers.Input(shape=(2, pop_num))
A_sqr = tf.math.multiply(A, A)

A_split_x = tf.split(A_sqr, 2, axis=1, name='split')[0]
A_split_y = tf.split(A_sqr, 2, axis=1, name='split')[1]

A_sum = tf.math.add(A_split_x, A_split_y)
A_norm = tf.linalg.normalize(A_sum)

# A_slice_a = tf.slice(A_sqr, [0,0,0], [1,2,1], name=None) 
# A_slice_b = tf.slice(A_sqr, [0,1,0], [1,2,1], name=None) 
# A_ave = layers.AveragePooling2D(pool_size=(2))(A)


In [82]:
fitness_function = tf.keras.Model(A, A_norm)

print(fitness_function.predict(tf.constant([[[1,2,3,4],[5,6,7,8]]])))

# Das Netzwert berechnet die Abstandsnorm der x,y Koordinaten zum ursprung.
# Für die Nutzung als Fitness Function ist der Output des Models normiert.

(array([[[0.23695184, 0.3645413 , 0.5285849 , 0.7290826 ]]], dtype=float32), array([[[109.72694]]], dtype=float32))


### Reproduktion (in Matrix-Representation - Tensorflow)

#### Option 1:

In [ ]:
for i in range(generations):
    fitness_function(final)

    final_coordinates = experience(population)               # Array contains final coordinates of all agents in the population (2 x P)
    fitness_function(final_coordinates)

    # The 50'th best (closest) agents get to repopulate and cross mutate
    # Mutation: Addition of random tensors and agent's model's weights
    # Crossmutation: Split tensors of model's weights and concat slices of two different agents

What if we could implement the selection and reproduction process as an optimizer?

Anstonsten könnte man den Selectionsprozess wie folgt implementieren:

In [ ]:
list_of_50_best_agents = None
list_of_50_best_agents = .shuffel(list_of_50_best_agents)


old_agent = list_of_50_best_agents[-1]
for agent in list_of_50_best_agents[:-2]:
    # split agent.model.weights and concatanate with old_agent.
    # Create both combinations.
    
    old_agent.model = None
    agent.model = None

    old_agent = agent

new_agents = [list_of_50_best_agents] * 2

#### Option 2:

Geringere Anzahl an Generations dafür aber Training der Agents mit tf.keras.Model.fit()

## Kann ich als loss den mse von den target koordinaten und der letzten position des agents nehmen?

In [103]:
def get_life_model(agent_model, simulation_steps=10):
    
    # model_life = tf.keras.Sequential()

    A = layers.Input(shape=(L,W,2), name='image_enviroment')
    K = layers.Input(shape=2, name='agent_position')

    enviroment = get_enviroment_model(A, agent_model.output, K)
    enviroment.trainable = False

    agent_motions = []

    agent_motion = agent_model(A)
    agent_motions.append(agent_motion)
    (A_new, K_new) = enviroment((A, agent_motion, K))

    for experience in range(simulation_steps - 1):
        agent_motion = agent_model(A_new)
        agent_motions.append(agent_motion)
        (A_new, K_new) = enviroment((A_new, agent_motion, K_new))

    model_life = tf.keras.Model((A, K), (A_new, K_new, agent_motions), name='one_life_experience')
    return model_life

In [104]:
# Create Random Training Data:
position = np.array([[1,1]])
target_position = np.array([[3,3]])

image = np.zeros(shape=(L,W,2))
image[position[:,0], position[:,1], 0] = 1                      # Place the agent in the image of the enviroment
image[target_position[:,0], target_position[:,1], 1] = 1        # Place the target in the image of the enviroment
image = tf.constant([image])


# Create Model in order to training an agent:
agent_nn = get_agent_model()
model_life = get_life_model(agent_nn, 5)
model_life.summary()
model_life.predict((image, position))

# print('Before motion:')
# print(np.swapaxes(np.squeeze(image), 0, 2))
# print('Position:', position)

print('\nAfter motion:')
# print(np.squeeze(model_life.predict((image, position))[0]))
print(np.swapaxes(np.squeeze(model_life.predict((image, position))[0]), 0, 2))
print('Position:', model_life.predict((image, position))[1])
print('Motions:', np.array(model_life.predict((image, position))[2]))

Model: "one_life_experience"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_enviroment (InputLayer)  [(None, 10, 10, 2)]  0           []                               
                                                                                                  
 agent_nn (Functional)          (None, 4)            804         ['image_enviroment[0][0]',       
                                                                  'Enviroment[0][0]',             
                                                                  'Enviroment[1][0]',             
                                                                  'Enviroment[2][0]',             
                                                                  'Enviroment[3][0]']             
                                                                                

In [26]:
def life(agent, days_in_life):
    random_target_positions = tf.random.constant()
    test_random_target_positions = tf.random.constant()
    model_life = get_life_model(agent.model)
    model_life.fit(random_target_positions, random_target_positions, epochs=days_in_life)
    
    return model_life.evaluate(test_random_target_positions)[index_of_the_accuracy]

In [ ]:
for i in range(generations):
    # Train for T epochs corresponding to the experience of one life-time
    
    for i, agent in population.agents:
        fitness_scores[i] = life(agent, days_in_life=days_in_life)


    # The 50'th best (closest) agents get to repopulate and cross mutate
    # Crossmutation: Split tensors of model's weights and concat slices of two different agents